## Text Classification with RNN

In [0]:
!pip install --upgrade grpcio
!pip install tensorflow==2.0
!pip install tfds-nightly

In [0]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

In [0]:
tfds.disable_progress_bar()
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

***tfds.load()*** returns tf.data.Dataset and tf.core.DatasetInfo instances. 

[tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) contains elements that have the same nested structure. The individual components of the structure can be of any type representable by [tf.TypeSpec](https://www.tensorflow.org/api_docs/python/tf/TypeSpec). tfds.load() returns all the splits (train, validation, and test) of the dataset if none is specified.

[tf.core.DatasetInfo](https://www.tensorflow.org/datasets/api_docs/python/tfds/core/DatasetInfo) documents the dataset info like name, version, features, url, citations, metadata etc. tfds.load() returns the information of the entire dataset if with_info=True, irrespective of the split requested. Split specific information is available in the splits attribute of the instance.




*tfds* includes a set of *TextEncoders* and *Tokenizers*.

[**TextEncoder**](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/text/TextEncoder) class in TensorFlow: Is an abstract base class for conversion between integers and text. Since text data has variable length and requires padding, ID 0 is always reserved for padding.

It has *vocab_size* as an attribute. vocab_size includes ID 0.

Method *encode()* encodes text into a list of integers. It never returns ID 0, and all IDs are always 1+.
Method *decode()* decodes a list of integers into text. It drops 0 in the input IDs.


[**SubwordTextEncoder**](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/text/SubwordTextEncoder) is an invertible TextEncoder using word pieces with a byte-level fallback. This encoding is fully invertible as all out-of-vocab wordpieces are byte-encoded. 

It contains *vocab_list* attribute which contains a list of subwords for the vocabulary. An underscore at the end of the vocabulary indicates the end of the word. Underscores in the interior of subword are disallowed and should be used with escape sequence.

**The dataset *info* includes the SubTextEncoder.**

In [3]:
print("info features: ", info.features)
encoder = info.features["text"].encoder
print("\n Vocabulary size: ", encoder.vocab_size)
print("\n Sample Subwords", encoder.subwords[10:20])

info features:  FeaturesDict({
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(None,), dtype=tf.int64, encoder=<SubwordTextEncoder vocab_size=8185>),
})

 Vocabulary size:  8185

 Sample Subwords ['in_', 'I_', 'that_', 'this_', 'it_', ' /><', ' />', 'was_', 'The_', 'as_']


As discussed above, the encoding is invertible.

In [4]:
sample_str = "IMDB Review Classification"
print("Original string: ", sample_str)

encoded_str = encoder.encode(sample_str)
print("Encoded string is: ", encoded_str)

decoded_str = encoder.decode(encoded_str)
print("Decoded string si: ", decoded_str)

for index in encoded_str:
    print("%s ----> %s"%(index, encoder.decode([index])))

Original string:  IMDB Review Classification
Encoded string is:  [5469, 7997, 2432, 3621, 739, 656, 2369, 1395, 3203, 757]
Decoded string si:  IMDB Review Classification
5469 ----> IM
7997 ----> D
2432 ----> B 
3621 ----> Rev
739 ----> ie
656 ----> w 
2369 ----> Cla
1395 ----> ssi
3203 ----> fic
757 ----> ation


### Preparing the dataset for training

We create batches of the encoded strings. **padded_batch** is used to zero pad the sequences to the length of the longest sequence in the batch. It combines consecutive elements of the dataset into padded batches.

In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [10]:
print(dir(train_dataset))
# fills a buffer with buffer_size elements, then randomly samples elements from this buffer, replacing the selected elements with new elements. For perfect shuffling, a buffer size greater than or equal to the full size of the dataset is required.
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
# output_shapes returns the shape of each component of an element of this dataset.
train_dataset = train_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_dataset))

test_dataset = test_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_dataset))

['_GeneratorState', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_add_variable_with_custom_getter', '_apply_options', '_as_serialized_graph', '_batch_size', '_checkpoint_dependencies', '_component_metadata', '_consumers', '_deferred_dependencies', '_drop_remainder', '_flat_shapes', '_flat_structure', '_flat_types', '_from_components', '_functions', '_gather_saveables_for_checkpoint', '_graph', '_graph_attr', '_handle_deferred_dependencies', '_has_captured_ref', '_input_dataset', '_inputs', '_is_graph_tensor', '_list_extra_dependencies_for_serialization', '_list_functions_fo

### Model

**[Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)** inherits from the base class Model, and groups a linear stack of layers into an object with training and inference features. 

**[Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)** inherits from the base class Layer and turns positive integer indices into vectors of fixed size. Takes as argument, input_dim (size of the vocabulary) and output_dim (dimension of the dense embedding).

In [14]:
n_hidden = 5

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=encoder.vocab_size, output_dim=64))
model.add(tf.keras.layers.SimpleRNN(units=n_hidden, activation='softmax'))
model.summary()

# output_seqs, states = rnn.static_rnn(basic_cell, train_dataset, dtype=tf.float32)


# TODO: Read about layers: tf.keras.layers.RNN, tf.keras.layers.LSTM, tf.keras.layers.GRU

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          523840    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 5)                 350       
Total params: 524,190
Trainable params: 524,190
Non-trainable params: 0
_________________________________________________________________


![alt text](https://github.com/shruti-singh/DL-NLP/blob/master/images/SingleRNN.png?raw=1)